In [2]:
import os

In [3]:
%pwd

'e:\\MLProject\\chicken-disease\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\MLProject\\chicken-disease'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [7]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,   
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        
        return data_ingestion_config

In [9]:
import os
import urllib
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        logger.info("Starting data download...")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(
                f"Data downloaded successfully! and saved at {self.config.local_data_file} "
                f"and the file size is {get_size(Path(filename))}"
            )
        else:
            logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        logger.info("Starting data extraction...")
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

    logger.info("Data ingestion completed successfully.")
except Exception as e:
    raise e 


[2026-01-25 14:06:05,192 - CNNClassifier - INFO - yaml file: config\config.yaml loaded successfully]
[2026-01-25 14:06:05,195 - CNNClassifier - INFO - yaml file: params.yaml loaded successfully]
[2026-01-25 14:06:05,196 - CNNClassifier - INFO - Directory created at: artifacts]
[2026-01-25 14:06:05,199 - CNNClassifier - INFO - Directory created at: artifacts/data_ingestion]
[2026-01-25 14:06:05,201 - CNNClassifier - INFO - Starting data download...]
[2026-01-25 14:06:07,758 - CNNClassifier - INFO - File size for artifacts\data_ingestion\data.zip is 11345 KB]
[2026-01-25 14:06:07,758 - CNNClassifier - INFO - Data downloaded successfully! and saved at artifacts/data_ingestion/data.zip and the file size is ~11345 KB]
[2026-01-25 14:06:07,758 - CNNClassifier - INFO - Starting data extraction...]
[2026-01-25 14:06:08,448 - CNNClassifier - INFO - Data ingestion completed successfully.]
